### <h1 align="center"> Hotel booking demand</h1>
<h2 align="lef" style='color:red' > Upvote my work if you found it useful</h2>
<h2 align="lef" style='color:red'>Wait a few seconds until the notebook is ready</h2>
<img src='https://www.h-hotels.com/_Resources/Persistent/dfd5c8fd7ab85af2515443995a2497e88bd1151b/aussenansicht-nacht-03-h4-hotel-berlin-alexanderplatz-2400x1113-1356x629.jpg' width=1000 hight=100><br>
<b>You can also see:</b><br>
<li><b><a href='https://www.kaggle.com/alaasedeeq/data-exploration-and-visualization-uber-data'>Data exploration and visualization(Uber Data)</a><br>
<li><b><a href='https://www.kaggle.com/alaasedeeq/suicide-rates-visualization-and-geographic-maps/edit/run/53135916'>Suicide Rates visualization and Geographic maps</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/superstore-data-analysis-with-plotly-clustering'>Superstore Data Analysis With Plotly(Clustering)</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/superstore-analysis-with-cufflinks-and-pandas'>Superstore Analysis With Cufflinks and pandas</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/european-soccer-database-with-sqlite3'>Chinook Questions with sqlite</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/chinook-questions-with-sqlite'>European soccer database with sqlite3</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/chinook-questions-with-sqlite'>Chinook data questions with sqlite3</a>

In [ ]:
#libraries for analysis and visualizatihon
import os
import pandas as pd 
import numpy as np
from collections import Counter
# import sort_dataframeby_monthorweek as sd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import plotly 
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
from cufflinks.offline import go_offline
import warnings

#libraries for ML
import sklearn
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import eli5

# %matplotlib notebook
warnings.filterwarnings('ignore')
init_notebook_mode()
go_offline(connected=False)
basemap=folium.Map()
colors = px.colors.qualitative.Prism

In [ ]:
#show existing files
os.listdir('../input/hotel-booking-demand')

In [ ]:
#read the data
df = pd.read_csv('../input/hotel-booking-demand/hotel_bookings.csv')
#make a cop of the original data
original_df = df.copy()
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df[df.select_dtypes(exclude='object').columns].describe().\
style.background_gradient(axis=1,cmap=sns.light_palette('green', as_cmap=True))

In [ ]:
# check for missing values
df.isnull().sum()

In [ ]:
#null percentage for each column

print(round(100*(df.isnull().sum()/len(df.index)),2))

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(df.isnull(),cmap='viridis');

In [ ]:
df[df.select_dtypes(exclude='object').columns].describe().\
style.background_gradient(axis=1,cmap=sns.light_palette('green', as_cmap=True))

<h3> Conclusion</h3><br>
<b>We have missing values in Country, agent, company and childeren columns, but the most missing values are in company columns

In [ ]:
#Correlation Map
corr = df.corr()
corr.iplot(kind='heatmap',colorscale='Blues',hoverinfo='all',
           layout = go.Layout(title='Correlation Heatmap',titlefont=dict(size=20)))

<h3> Conclusion</h3><br>
<b>The columns have highest correlation values with is_canceled column are:</b>
<li>lead_time wiht <b>0.293123</b>
<li>total_of_special_requests with <b>-0.234658</b>  
<li>required_car_parking_spaces with <b>0.195498</b>
<li>previous booking_changes with <b>-0.144381</b>
<li>previous_cancellations with <b>0.110133</b>

# Cleaning The Data

><B>Dealing With The Missing Values

In [ ]:
#replace missing value in country column with 'unknown'
df['country'] = df['country'].fillna('unknown')

#replace missing value in company column with 0
df['company'] = df['company'].fillna(0)

#replace missing value in childern column with 0
df['children'] = df['children'].fillna(0)

#replace missing value in agent column with 0
df['agent'] = df['agent'].fillna(0)

In [ ]:
#converting date column to datestamp
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

#Lets create a new columns with the total number of nights stayed at the hotel
df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

#Lets create a new columns with the room pric for person (assuming the guest is an adult or a children)
df['price'] = df['adr'].div(df['adults'] + df['children'])

#lets deal with the data without the rows contain a zero guests
idx = df[(df['children']+df['adults']+df['babies'])==0].index #indecies of rows have zero guests
df = df[~df.index.isin(idx)]
df.shape

# Exploratory Data Analysis

In [ ]:
canceled = df[df['is_canceled']==1]
not_canceled = df[df['is_canceled']==0]

print('Canceled : {} ({} %)\nNot Canceled : {} ({} %)'.\
      format(canceled.shape[0],round(canceled.shape[0]/(canceled.shape[0]+not_canceled.shape[0])*100,2),
             not_canceled.shape[0],round(canceled.shape[0]/(canceled.shape[0]+not_canceled.shape[0])*100,2)))

In [ ]:
df['hotel'].unique()

In [ ]:
#we have to hotels ('Resort Hotel', 'City Hotel')
resort_hotel = df[(df['hotel']=='Resort Hotel')&(df['is_canceled']==0)]
city_hotel = df[(df['hotel']=='City Hotel')&(df['is_canceled']==0)]

print('Resort Hotel : {} ({} %)\nCity Hotel : {} ({} %)'.\
      format(resort_hotel.shape[0],round(resort_hotel.shape[0]/(resort_hotel.shape[0]+city_hotel.shape[0])*100,2),
             city_hotel.shape[0],round(city_hotel.shape[0]/(resort_hotel.shape[0]+city_hotel.shape[0])*100,2)))

In [ ]:
pd.concat([city_hotel,resort_hotel])['hotel'].iplot(kind='histogram',title='Total number of booking for each hotel')

In [ ]:
temp = pd.DataFrame([len(df) for df in [city_hotel,resort_hotel]])\
.rename(index={0:'City Hotel',1:'Resort Hotel'},columns={0:'count'}).reset_index()

layout = cf.Layout(title='Total number of booking for each hotel',titlefont=dict(size=20),
                   xaxis=dict(title=''),yaxis=dict(title=''))

temp.iplot(kind='pie',hole=0.5,pull=0.03,textinfo='label+percent',
           colors=colors,
           values='count',labels='index',
           layout=layout)

In [ ]:
temp = pd.DataFrame()
temp['City Hotel'] = df[df['hotel']=='City Hotel']['is_canceled'].value_counts().values
temp['Resort Hotel'] = df[df['hotel']=='Resort Hotel']['is_canceled'].value_counts().values
temp = temp.T.rename(columns={0:'Not Canceled',1:'Canceled'})

fig = px.bar(temp,
             barmode='group',
             x = temp.index,
             y = [c for c in temp.columns],
             template = 'plotly_white',
             color_discrete_sequence = colors,
             title='Grouped Bar Plot for total number booking for each hotel <br>With (Canceled or Not Canceled) as color')

fig['layout'].update(titlefont=dict(size=20),
                     xaxis=dict(title='Hotel',titlefont=dict(size=18)),
                     yaxis=dict(title='Number of booking',titlefont=dict(size=18)))
fig.show()

<h3>Conclusion</h3><br>
<b>City Hotel hase more guests than Resort Hotel and also has higher cancelation 

In [ ]:
temp = df['arrival_date_year']

temp.iplot(kind='histogram',
           color=colors,
           bargap=0.2,
           title='Total guests for each year')

In [ ]:
temp = pd.DataFrame(index=df.groupby('customer_type').sum().index)

for hotel in df['hotel'].unique():
    temp[hotel] = df[df['hotel']==hotel]['customer_type'].value_counts()
    
temp = temp.sort_values('City Hotel',ascending=False)
temp.iplot(kind='bar',
           colors=colors,
           title='Total guests for  each hotel<br>Grouped by customer type')

In [ ]:
temp = pd.DataFrame(index=df.groupby(df['total_nights']).sum().index)

for hotel in df['hotel'].unique():
    temp[hotel] = df[df['hotel']==hotel]['total_nights'].value_counts()
    
temp = temp.sort_values('City Hotel',ascending=False)
temp.iplot(kind='bar',title='Total guests for  each hotel<br>Grouped by total nights stayed in each hotel')

In [ ]:
temp = pd.DataFrame(index=df.groupby(df['arrival_date_month']).sum().index)

for hotel in df['hotel'].unique():
    temp[hotel] = df[df['hotel']==hotel]['arrival_date_month'].value_counts()
    
temp.iplot(kind='line',title='Total guests for  each hotel<br>By arrival date month in each hotel')

#Try useing this to sort month indexn berfore plottting
# temp = temp.sort_values('City Hotel',ascending=False).reset_index()

# temp = sd.Sort_Dataframeby_Month(df=temp,monthcolumnname='arrival_date_month').set_index('arrival_date_month')

In [ ]:
temp = pd.DataFrame(index=df.groupby(df['market_segment']).sum().index)

for hotel in df['hotel'].unique():
    temp[hotel] = df[df['hotel']==hotel]['market_segment'].value_counts()
    
temp = temp.sort_values('City Hotel',ascending=False)
temp.iplot(kind='bar',title='Total guests for  each hotel<br>Grouped by arrival date month in each hotel')

In [ ]:
temp = pd.DataFrame(index=df.groupby('arrival_date_month').mean().index)

temp['City Hotel Canceled'] = df[(df['hotel']=='City Hotel')&(df['is_canceled']==1)]['arrival_date_month'].value_counts()
temp['City Hotel Not Canceled'] = df[(df['hotel']=='City Hotel')&(df['is_canceled']==0)]['arrival_date_month'].value_counts()

temp.iplot(kind='bar',title='Number of Canceled and Non-Canceled reservation <br>Over the year for City Hotel')

In [ ]:
temp = pd.DataFrame(index=df.groupby('arrival_date_month').mean().index)

temp['Resort Hotel Canceled'] = df[(df['hotel']=='Resort Hotel')&(df['is_canceled']==1)]['arrival_date_month'].value_counts()
temp['Resort Hotel Not Canceled'] = df[(df['hotel']=='Resort Hotel')&(df['is_canceled']==0)]['arrival_date_month'].value_counts()

temp.iplot(kind='bar',title='Number of Canceled and Non-Canceled reservation<br>Over the year for Resort Hotel')

In [ ]:
temp = pd.DataFrame(index=df['arrival_date_month'].unique())

for year in df['arrival_date_year'].unique():
    temp[year] = df[df['arrival_date_year']==year]['arrival_date_month'].value_counts()
    

#Try useing this to sort month indexn berfore plottting
# temp = temp.reset_index()
# temp = sd.Sort_Dataframeby_Month(df=temp,monthcolumnname='index').set_index('index')

temp.iplot(kind='bar',
          title='Total guests over the year for each year<br>(2015-2016-2017)')

><h1>Country

<b>Computing the number of guest wich did not cancel the reservation for each country
>><b>Bar Plot

In [ ]:
country_df = df['country'].value_counts().to_frame().rename(columns={'country':'Total Booking'})
country_df.head(10).style.background_gradient(cmap='PuBu')

In [ ]:
temp = df[df['is_canceled']==0]['country'].value_counts()

temp.iplot(kind='bar',title='Bar Plot for total Guests by Countries')

In [ ]:
temp = pd.DataFrame()

temp['City Hotel'] = pd.Series(df[(df['hotel']=='City Hotel')&(df['is_canceled']==0)]['country'].value_counts().values)
temp['Resort Hotel'] = pd.Series(df[(df['hotel']=='Resort Hotel')&(df['is_canceled']==0)]['country'].value_counts().values)

idx_number = max(len(temp['City Hotel']),len(temp['Resort Hotel']))

temp.set_index(df[(df['is_canceled']==0)]['country'].value_counts().index[:idx_number],inplace=True)


temp.iplot(kind='bar',title='Bar Plot for total Guests by Countries for each hotel')

>><b>Pie Plot

In [ ]:
temp = df[df['is_canceled']==0]['country'].value_counts().reset_index()

layout = cf.Layout(xaxis=dict(title=''),yaxis=dict(title=''),
                  title='Pie Plot For Guests Countries',
                  titlefont=dict(size=20),
                  height=700,width=1000)

temp.iplot(kind='pie',
           labels='index',
           values='country',
           textinfo='value+percent+label',
           layout=layout,pull=0.03,sort=True)

>><b>Map

In [ ]:
# show on map

temp = df['country'].value_counts().reset_index().rename(columns={'index':'country','country':'count'})

guest_map = px.choropleth(temp,
                          locations=temp['country'],
                          color=np.log(temp['count']), 
                          hover_name=temp['country'], 
                          color_continuous_scale=px.colors.sequential.Plasma,
                          title="Home country of guests")

guest_map.show()

<h3>Conclusion</h3><br>
<b>Most hotel guests in each hotel come from Portugal

### Price Analysis

In [ ]:
#Price Histogram 
df['price'].iplot(kind='histogram',title='Room price per night and person Distribution')

> <b>Lets See the effects of outlier and we will deal with it later

In [ ]:
#Detecting the outliers
def detect_outliers(df ,feature ,class_name ,target_feat):
    class_in_feature = df[df[feature]==class_name][target_feat]
    q25 , q75 = np.percentile(class_in_feature ,25) , np.percentile(class_in_feature ,75)
    iqr = q75 - q25
    threshold = 1.5
    lower ,upper = (q25 - threshold * iqr), (q75 + threshold * iqr)
    outliers = [x for x in class_in_feature if x < lower or x > upper]
    
    print(' Outliers for {} : {}'.format(class_name,len(outliers)))
    print(' Quartile_25 : {}'.format(q25),'\n','Quartile_75 : {}'.format(q75),'\n','IQR : {}'.format(iqr))
    print(' Cut Off: {}'.format(threshold*iqr))
    print(' {} Lower: {}'.format(target_feat ,lower))
    print(' {} Upper: {}'.format(target_feat ,upper))
    print('\n',50*'=','\n')

In [ ]:
detect_outliers(df,'hotel','City Hotel','price')
detect_outliers(df,'hotel','Resort Hotel','price')

<b>We can see the effect of outliers clearly in price distribution

In [ ]:
temp = df.copy()
temp.loc[temp['price'] > 200,'price'] = 200

data_1 = df['price'].reset_index().rename(columns=lambda x:x.capitalize() +' With Outliers')
data_2 = temp.rename(columns=lambda x:x.capitalize() +' Without Outliers')

data_1['Price With Outliers'].iplot(kind='hist',
                               subplots=True,
                               horizontal_spacing=.1,
                               fill=True,
                               subplot_titles=True,
                               title='Room price per night and person Distribution With Outliers')
       
data_2['Price Without Outliers'].iplot(kind='hist',
                               subplots=True,
                               horizontal_spacing=.1,
                               fill=True,
                               subplot_titles=True,
                               title='Room price per night and person Distribution Without Outliers')       

<B>We can see the effect of outliers on the data clearly 
<p>let's proceed our EDA

In [ ]:
temp = pd.DataFrame(index=df.index)

for year in df['arrival_date_year'].unique():
    temp[year] = pd.Series(df[df['arrival_date_year']==year]['price'])
    
temp.iplot(kind='box',
          title='Room price per night and person for each year<br>(2015-2016-2017)')

In [ ]:
temp = df.groupby('country').mean()['price'][:20].to_frame().sort_values('price',ascending=False)

temp.iplot(kind='bar',colorscale='set1',title='Bar Plot for the mean Room price per night and person<br>For top ten countries')

In [ ]:
# show on map

temp = df.groupby('country')['price'].mean().to_frame().sort_values('price',ascending=False).reset_index()

guest_map = px.choropleth(temp,
                          locations=temp['country'],
                          color=np.log(temp['price']), 
                          hover_name=temp['country'], 
                          color_continuous_scale=px.colors.sequential.Plasma,
                          title="Room price per night and person<br>For each country")

guest_map.show()
# temp

<h3>Conclusion</h3><br>
<b>We found that there are only two rows have country Antarctica, But it stil the highest mean price !

In [ ]:
temp = df.groupby('reserved_room_type')['price'].mean().to_frame().sort_values('price',ascending=False)

temp.iplot(kind='bar',colorscale='set1',title='Bar Plot for the  Room mean price per night and person<br>For each reserved room type')

<h3>Conclusion</h3><br>
<b>We found that L-Type Room has the highest mean price !

In [ ]:
temp = df.groupby('customer_type')['price'].mean().to_frame().sort_values('price',ascending=False)

temp.iplot(kind='bar',colorscale='set1',title='Bar Plot for the  Room mean price per night and person<br>For each reserved room type')

<h3>Conclusion</h3><br>
<b>We found that Transient guest pay the most !

In [ ]:
temp = df.groupby('market_segment')['price'].mean().to_frame().sort_values('price',ascending=False)

temp.iplot(kind='bar',colorscale='set1',title='Bar Plot for the  Room mean price per night and person<br>For each reserved room type')

<b>We find that the Aviation segment has low guests count, but also has the highest price

In [ ]:
temp = pd.DataFrame(index=df.groupby('arrival_date_month').mean().index)

temp['City Hotel'] = df[df['hotel']=='City Hotel'].groupby('arrival_date_month').mean()['price'].to_frame()

temp['Resort Hotel'] = df[df['hotel']=='Resort Hotel'].groupby('arrival_date_month').mean()['price'].to_frame()

# temp = temp.reset_index()
# temp = temp.reset_index()
# temp = sd.Sort_Dataframeby_Month(df=temp,monthcolumnname='arrival_date_month').set_index('arrival_date_month')

temp.iplot(kind='line',title='Room mean price per night and person over the year')

In [ ]:
temp = pd.DataFrame(index=df.groupby('reserved_room_type').sum().index)

for seg in df['market_segment'].unique():
    temp[seg] = df[df['market_segment']==seg].groupby('reserved_room_type').mean()['price']
    
temp.T.iplot(kind='bar',title='Room mean price per night and person<br>By market segment and reserved room type')

### Market Segment Analysis

In [ ]:
temp = df.groupby('market_segment').sum()
temp = temp[['lead_time','previous_cancellations','previous_bookings_not_canceled','agent','adr','total_of_special_requests']]
temp.head().style.background_gradient(cmap='PuBu')

In [ ]:
temp = df['market_segment'].value_counts().to_frame().rename(columns={'market_segment':'Total Guests'}).reset_index()

temp.iplot(kind='pie',
           labels='index',
           values='Total Guests',
           textinfo='label+percent',
           title='Total Guests for each market segment',
           pull=0.03,sort=True)

In [ ]:
temp = pd.DataFrame(index=df.hotel.unique())
for seg in df['market_segment'].unique():
    temp[seg] = df[df['market_segment']==seg]['hotel'].value_counts()
    
temp.T.iplot(kind='bar',title='Total number of guests for each hotel by market segment')

In [ ]:
temp = pd.DataFrame(index=df['reserved_room_type'].unique())
for seg in df['market_segment'].unique():
    temp[seg] = df[df['market_segment']==seg]['reserved_room_type'].value_counts()
    
temp.T.iplot(kind='bar',title='Total number of guests for each hotel by market segment')

In [ ]:
temp = pd.DataFrame(index=df.index)

for seg in df['market_segment'].unique():
    temp[seg] = pd.Series(df[df['market_segment']==seg]['price'].values)
    
temp.iplot(kind='box',
           mean=True,
           title='Price Distribution for each market segment')

In [ ]:
temp = pd.DataFrame(index=df.index)

for seg in df['market_segment'].unique():
    temp['Resort Hotel '+seg] = pd.Series(df[(df['market_segment']==seg)&(df['hotel']=='Resort Hotel')]['total_nights'].values)
    temp['City Hotel '+seg] = pd.Series(df[(df['market_segment']==seg)&(df['hotel']=='City Hotel')]['total_nights'].values)

temp.iplot(kind='box',
           mean=True,
           xTitle='Market Segment',
           yTitle='Total Nights',
           title='Total Nights Stayed Distribution<br>For each market segment by hotel')

<h3> Conclusion</h3><br>
<b>It can be seen that most of the groups are normal distributed, some of them have high skewness. Looking at the distribution, most people do not seem to prefer to stay at the hotel for more than 1 week. But it seems normal to stay in resort hotels for up to 12-13 days
It is obvious that when people go to resort hotels, they prefer to stay more.

### Meal Analysis

In [ ]:
temp = df['meal'].value_counts().to_frame().reset_index()

layout = cf.Layout(title='Total booking meals',
                   titlefont=dict(size=25),
                  xaxis=dict(title=''),yaxis=dict(title=''))

temp.iplot(kind='pie',
           labels='index',
           values='meal',
           hole=0.5,pull=0.02,
           colors=colors,
           layout=layout,
           textinfo='label+percent')

In [ ]:
temp = pd.DataFrame(index=df.index)

for meal in df['meal'].unique():
    temp[meal] = pd.Series(df[df['meal']==meal]['price'].values)
    
temp.iplot(kind='box',
           mean=True,
           title='Price Distribution<br>For each Meal')


<h3>Conclusion</h3><br>
<b>There is a big difference in the Bed&Breakfast (BB) category and the others. Almost 80% of bookings reserved for Bed & Breakfast(BB).

<h1>Outliers</h1>

<b>Lets see the effect of outliers in the data

In [ ]:
cols = ['price','total_nights','children','lead_time','days_in_waiting_list',
        'previous_cancellations','adults','previous_bookings_not_canceled']

df[cols].iplot(kind='hist',
               subplots=True,
               horizontal_spacing=.2,
               fill=True,
               dimensions=(1000,1000),
               subplot_titles=True,
               title='Data Distribution')

<b>Dealing With the outliers in these columns

In [ ]:
df_cln = df.copy()

cols = ['price','total_nights','children','lead_time','days_in_waiting_list',
        'previous_cancellations','adults','previous_bookings_not_canceled']

df_cln.loc[df_cln['price'] > 200,'price'] = 200
df_cln.loc[df_cln['total_nights'] > 15,'LeadTime'] = 15
df_cln.loc[df_cln['children'] > 3,'children'] = 3
df_cln.loc[df_cln['lead_time'] > 500,'lead_time'] = 500
df_cln.loc[df_cln['adults'] > 4,'adults'] = 4
df_cln.loc[df_cln['previous_bookings_not_canceled'] > 0,'previous_bookings_not_canceled'] = 1
df_cln.loc[df_cln['previous_cancellations'] > 0,'previous_cancellations'] = 1
df_cln.loc[df_cln['days_in_waiting_list'] > 0,'days_in_waiting_list'] = 1

df_cln[cols].iplot(kind='hist',
               subplots=True,
               horizontal_spacing=.2,
               fill=True,
               dimensions=(1000,1000),
               subplot_titles=True,
               title='Data Distribution Without outliers')

### <b><h2 align="center" style='color:black' > Thanks</h2>
### <li>Machine learning part will be added soon</li>